In [99]:
import praw
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm_notebook
import psaw
import sklearn as skl
from sklearn import feature_extraction
pd.set_option("display.max_columns",None)

In [96]:
df_all = pd.read_csv("src/data/original_data/AITA_all.csv")
df_ass = pd.read_csv("src/data/original_data/AITA_ass.csv")
df_not = pd.read_csv("src/data/original_data/AITA_not.csv")
print(df_all.shape, df_ass.shape, df_not.shape)

(16000, 33) (4421, 33) (10000, 33)


In [108]:
df_ass[df_ass["selftext"] == "[deleted]"].shape

(378, 33)

In [97]:
# removing samples with no selftext (post is deleted)
df_all = df_all[df_all["selftext"] != "[deleted]"]
df_all.reset_index(drop=True, inplace = True)
df_all.shape

(13758, 33)

In [98]:
# select samples from df_all that is either tagged as not a asshole or a asshole
mask = (df_all["link_flair_css_class"] == "not") | (df_all["link_flair_css_class"] == "ass")
df_all = df_all[mask]
df_all.reset_index(drop = True, inplace = True)
df_all.shape

(6503, 33)

In [100]:
# preprocess text
ass_selftext = df_ass["selftext"].values
not_selftext = df_not["selftext"].values

In [103]:
print(ass_selftext[0])

For context, I did not kick them off the machine. I was waiting for the Calf Raise Machine where a woman was currently working in. Right before I go to approach her, to ask if I can rotate in, her friend tells her to look behind her because she can tell that I am waiting. I pose my question, "Do you mind if I rotate in?". The woman who was initially on the machine sternly says, "No. Me and her are working out here." I told her, "Alright, but do you mind if we rotate?" Again, she says no with the same level of confidence only adding this time that they are "Trying to get out of here." I was a little taken aback by this so I responded in kind, "So am I." She refuses to let me rotate in the machine and this essentially starts a argument where she insists that I can't use the machine until she and her friend are done. I tell her that you can't just claim a machine and deny other people from using it, so I tell her that instead of all this arguing, I could have already used the machine. So,

In [ ]:

cv_ass = feature_extraction.text.CountVectorizer()
cv_not = feature_extraction.text.CountVectorizer()